In [1]:
import orekit

ModuleNotFoundError: No module named 'orekit'

In [ ]:
vm = orekit.initVM()

In [ ]:
from orekit.pyhelpers import setup_orekit_curdir

setup_orekit_curdir() # looks for orekit-data.zip

# grab two line elements from celestrak:
# https://celestrak.com/NORAD/elements/active.txt
iss_tle = [
    "1 25544U 98067A   20168.72989583  .00000038  00000-0  87409-5 0  9992",
    "2 25544  51.6465 350.7306 0002347  64.4399 115.7370 15.49444406231948"
]

In [ ]:
from org.orekit.propagation.analytical.tle import TLE, TLEPropagator

tle = TLE(iss_tle[0], iss_tle[1])

import math

epoch = tle.getDate()
print("date: {:s}, eccentricity: {:.4f}, inclination: {:.2f}".format(
    epoch.toString(),
    tle.getE(),
    math.degrees(tle.getI())
))

In [ ]:
from org.orekit.propagation.analytical.tle import TLEPropagator

propagator = TLEPropagator.selectExtrapolator(tle)

coords = propagator.getPVCoordinates(epoch)
position = coords.getPosition()
frame = propagator.getFrame()
print("frame: {:s}, x: {:.2f}, y: {:.2f}, z: {:2f}".format(
    frame.getName(),
    position.getX(),
    position.getY(),
    position.getZ()
))

In [ ]:
from org.orekit.frames import FramesFactory

# convert to an earth-centered inertial frame
eci = FramesFactory.getEME2000()
teme_to_eci_xf = frame.getTransformTo(eci, epoch)
eci_position = teme_to_eci_xf.transformPosition(position)
print("frame: {:s}, x: {:.2f}, y: {:.2f}, z: {:2f}".format(
    eci.getName(),
    eci_position.getX(),
    eci_position.getY(),
    eci_position.getZ()
))

In [ ]:
from org.orekit.utils import IERSConventions

# convert to an earth-centered fixed frame
ecef = FramesFactory.getITRF(IERSConventions.IERS_2010, True)
teme_to_ecef_xf = frame.getTransformTo(ecef, epoch)
ecef_position = teme_to_ecef_xf.transformPosition(position)
print("frame: {:s}, x: {:.2f}, y: {:.2f}, z: {:2f}".format(
    ecef.getName(),
    ecef_position.getX(),
    ecef_position.getY(),
    ecef_position.getZ()
))


In [ ]:
from org.orekit.bodies import OneAxisEllipsoid
from org.orekit.utils import Constants

# convert to geodetic coordinates
earth = OneAxisEllipsoid(
    Constants.WGS84_EARTH_EQUATORIAL_RADIUS,
 	Constants.WGS84_EARTH_FLATTENING,
    ecef
)
geo_position = earth.transform(position, frame, epoch)
print("geo: lat: {:.2f}, lon: {:.2f}, alt: {:2f}".format(
    math.degrees(geo_position.getLatitude()),
    math.degrees(geo_position.getLongitude()),
    geo_position.getAltitude()
))


In [ ]:
from orekit.pyhelpers import absolutedate_to_datetime, datetime_to_absolutedate
import datetime

# do a simple propagation
lat = []
lon = []
time = absolutedate_to_datetime(tle.getDate())
end_time = time + datetime.timedelta(hours=2)
while time < end_time:
    time = time + datetime.timedelta(minutes=1)
    coords = propagator.getPVCoordinates(datetime_to_absolutedate(time))
    position = coords.getPosition()
    geo_position = earth.transform(position, frame, datetime_to_absolutedate(time))
    lat.append(math.degrees(geo_position.getLatitude()))
    lon.append(math.degrees(geo_position.getLongitude()))


In [ ]:
import matplotlib.pyplot as plt

# plot the results
plt.figure()
plt.plot(lon, lat, '.r')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.savefig('propagation.png')
